In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-27 11:35:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.09MB/s    in 0.5s    

2024-09-27 11:35:56 (2.09 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [31]:
import numpy as np

In [2]:
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()
print(f"length of text is:{len(text)}")


length of text is:1115394


In [3]:
#looking up the text
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
#finding Vocab size

chars=sorted(list(set(text)))
vocab_size=len(chars)
print(f'Vocab is:{"".join(chars)}') #first characted is next line character
print(f'Length of vocab is:{vocab_size}')

Vocab is:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of vocab is:65


In [16]:
#simple tokenizer

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l:"".join(itos[s] for s in l)


encoding=encode("Hi There")
decoding=decode(encoding)
print(f'Encoding the string: \'Hi There\':{encoding}')
print(f'Decoding the encoding:{decoding}')


# encoding=encode("Hi Ther~~~~~")  This fails because ~ is not present in vocab.
# decoding="".join(decode(encoding))
# print(f'Encoding the string: \'Hi Ther~~~~~\':{encoding}')
# print(f'Decoding the encoding:{decoding}')


Encoding the string: 'Hi There':[20, 47, 1, 32, 46, 43, 56, 43]
Decoding the encoding:Hi There


In [19]:
#Tokenize the entire dataset:
import torch
data=torch.tensor(encode(text), dtype=torch.long) #we use dtype as torch.long as torch embedding layers expect long datatype as input.
print([data.shape, data.dtype])
print(data[:1000])

[torch.Size([1115394]), torch.int64]
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
   

In [27]:
#train test split
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [26]:
#chunking or block size

block_size=8
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f'when input is {context} the target is: {target}')


#More clear understanding
# block_size=8
# x=text[:block_size]
# y=text[1:block_size+1]
# for t in range(block_size):
#     context=x[:t+1]
#     target=y[t]
#     print(f'when input is {context} the target is: {target}')

# when input is F the target is: i
# when input is Fi the target is: r
# when input is Fir the target is: s
# when input is Firs the target is: t
# when input is First the target is:  
# when input is First  the target is: C
# when input is First C the target is: i
# when input is First Ci the target is: t

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [36]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data=train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))  #generates random indices for given batch size batch size
    #data = "Hello, this is an example sentence." (a sequence of 37 characters).
    # block_size = 5 (we are interested in blocks of 5 characters).
    # batch_size = 3 (we want to generate 3 random starting points).
    # Length of data:
    # len(data) = 37

    # Maximum random integer:
    # len(data) - block_size = 37 - 5 = 32

    # So, torch.randint(32, (3,)) will generate 3 random integers between 0 and 31. These integers could represent the starting indices of 5-character blocks within the sequence.
    # torch.randint(32, (3,))
    # # Output: tensor([7, 15, 2])
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

# def get_batch(split):
#     data=text if split=="train" else val_data
#     ix = torch.randint(len(data)-block_size,(batch_size,))  #generates random indices for given batch size batch size
#     #data = "Hello, this is an example sentence." (a sequence of 37 characters).
#     # block_size = 5 (we are interested in blocks of 5 characters).
#     # batch_size = 3 (we want to generate 3 random starting points).
#     # Length of data:
#     # len(data) = 37

#     # Maximum random integer:
#     # len(data) - block_size = 37 - 5 = 32

#     # So, torch.randint(32, (3,)) will generate 3 random integers between 0 and 31. These integers could represent the starting indices of 5-character blocks within the sequence.
#     # torch.randint(32, (3,))
#     # # Output: tensor([7, 15, 2])
#     x=np.stack([data[i:i+block_size] for i in ix])
#     y=np.stack([data[i+1:i+block_size+1] for i in ix])
#     return x,y

# inputs:
# (4,)
# ['unkind b' 'opeful l' 'OLUMNIA:' 'ice over']
# outputs
# (4,)
# ['nkind br' 'peful la' 'LUMNIA:\n' 'ce overt']

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('outputs')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [53]:
#Bigram Lamguage Model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)  #learning parameter or layer

    def forward(self, idx, targets=None):
        logits=self.token_embedding_table(idx) #Batch, Time, Channel
        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            idx_next=torch.multinomial(probs,num_samples=1)
            idx=torch.cat((idx,idx_next), dim=1)
        return idx

model=BigramLanguageModel(vocab_size)
out,loss=model(xb,yb)
print(out.shape)
print(loss)
idx=torch.zeros((1,1),dtype=torch.long)
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [59]:
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3)
batch_size=32
for steps in range(15000):
    xb,yb=get_batch('train')
    logits,loss=model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.520535945892334


In [61]:
idx=torch.zeros((1,1),dtype=torch.long)
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Ancoces:
TRISTwoyorern cowononcamu, te d ce; nke t ss lshedlamure guser l har'
my: ch' ys nt INonde,
